# Scyclone-PyTorch
[![Generic badge](https://img.shields.io/badge/GitHub-Scyclone--PyTorch-9cf.svg)][github]
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)][notebook]
[![Paper](http://img.shields.io/badge/paper-arxiv.2005.03334-B31B1B.svg)][paper]  

Reimplmentation of voice conversion system "Scyclone" with PyTorch  
Author: [tarepan](https://github.com/tarepan)

notebook URL differ from original (`tarepan/Scyclone-PyTorch` => `tarepan/MachineLearningNotebooks`)

[github]:https://github.com/tarepan/Scyclone-PyTorch
[paper]:https://arxiv.org/abs/2005.03334
[notebook]:https://colab.research.google.com/github/tarepan/MachineLearningNotebooks/blob/main/Scyclone_PyTorch.ipynb

## Colab Check
Check
- Google Colaboratory runnning time
- GPU type
- Python version
- CUDA version

In [ ]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'
!head -n 1 /proc/driver/nvidia/gpus/**/information
!cat /usr/local/cuda/version.txt

## Setup

Activate notebook for log session **by hand**
```javascript
const refresher = setInterval(()=>{document.querySelector("colab-connect-button").click();console.log("clicked for long session");}, 1000*60*10);
```

Install package from `tarepan/Scyclone-PyTorch` private repository

In [ ]:
# Secure credential acquisition
## GitHub
import os
import getpass
if os.environ.get('GitHub_credentials') == None:
  user_name = "tarepan"
  GitHub_password = getpass.getpass()
  os.environ['GitHub_credentials'] = user_name + ':' + GitHub_password
# GoogleDrive
from google.colab import drive
drive.mount('/content/gdrive')

# repository install
!pip uninstall scyclonepytorch -y -q
!pip install git+https://$GitHub_credentials@github.com/tarepan/Scyclone-PyTorch -q

Prepare data transfer functions

In [ ]:
import threading
import time
from distutils.dir_util import copy_tree


def sleep_transfer_loop(source_path: str, target_path:str, interval_min: float):
  """
  Transfer source path tree (e.g. ongoing training log) to target path (e.g. Google Drive), then sleep specified time, then repeat it.
  """
  interval_sec = interval_min * 60
  time.sleep(interval_sec)
  copy_tree(source_path, target_path)
  print("copied.")
  sleep_transfer_loop(source_path, target_path, interval_min)

def gen_TS_loop(source_path: str, target_path:str, interval_min: float = 10):
  return lambda: sleep_transfer_loop(source_path, target_path, interval_min)

## Training

In [ ]:
# intermittent log transfer
source_path = "logs"
target_path = f"gdrive/My Drive/ML_results/Scyclone"
interval_min = 10
threading.Thread(target=gen_TS_loop(source_path, target_path, interval_min)).start()

# Start TensorBoard
%load_ext tensorboard
%tensorboard --logdir logs

!python -m scyclonepytorch.Scyclone_main --num_workers=0 # from scratch
# !python -m scyclonepytorch.Scyclone_main --num_workers=0 --checkpoint=logs/default/version_100/checkpoints/epoch=8999.ckpt

In [ ]:
!ls logs/default/

## Training Optimization

### whole

In [ ]:
# num_worker x pinmemory

!python -m scyclonepytorch.Scyclone_main --max_epochs=15 --num_workers=0
!python -m scyclonepytorch.Scyclone_main --max_epochs=15 --num_workers=1
!python -m scyclonepytorch.Scyclone_main --max_epochs=15 --num_workers=2
!python -m scyclonepytorch.Scyclone_main --max_epochs=15 --num_workers=4

!python -m scyclonepytorch.Scyclone_main --max_epochs=15 --num_workers=0 --no_pin_memory
!python -m scyclonepytorch.Scyclone_main --max_epochs=15 --num_workers=1 --no_pin_memory
!python -m scyclonepytorch.Scyclone_main --max_epochs=15 --num_workers=2 --no_pin_memory
!python -m scyclonepytorch.Scyclone_main --max_epochs=15 --num_workers=4 --no_pin_memory

### num_worker

In [ ]:
!python -m scyclonepytorch.Scyclone_main --max_epochs=5 --num_workers=0
!python -m scyclonepytorch.Scyclone_main --max_epochs=5 --num_workers=1
!python -m scyclonepytorch.Scyclone_main --max_epochs=5 --num_workers=2
!python -m scyclonepytorch.Scyclone_main --max_epochs=5 --num_workers=4

### pin_memory

In [ ]:
!python -m scyclonepytorch.Scyclone_main --max_epochs=5 --num_workers=2
!python -m scyclonepytorch.Scyclone_main --max_epochs=5 --num_workers=2 --no_pin_memory

### Profiling

In [ ]:
!python -m scyclonepytorch.Scyclone_main --profiler --max_epochs=5 --num_workers=2  --no_pin_memory # profile mode

In [ ]:
# # Usage stat
# ## GPU
# !nvidia-smi -l 3
# ## CPU
# !vmstat 5
# !top
```